# ReAct

⭐️ ReAct (Reasoning + Acting) ⭐️는 프린스턴 대학교 학자들이 구글 연구원들과 협력하여 개발한 프롬프트 엔지니어링 전략입니다. 이 전략은 ***LLM이 인간처럼 문제를 풀기 위해 먼저 생각하고, 그에 따라 행동***하는 방식을 모방합니다.

ReAct가 없는 모델의 경우 다음과 같은 문제가 발생할 수도 있습니다.
```
1. 사용자가 "안중근 의사랑 세종대왕이랑 이순신 장군 알려줘"라고 요청한 경우,
2. 일반적인 모델의 경우 안중근 의사, 세종대왕, 이순신 장군에 대한 정보를 하나씩 생성하여 보여줍니다.
3. 잘 생성하다가 안중근 의사, 세종대왕에 대한 정보만 생성하고, 이순신 장군에 대한 정보는 생성하지 않을 가능성이 존재합니다.
```

ReAct는 모델이 여러 외부 도구(검색 도구)를 사용할 수 있음을 전제로 하며, 도구의 결과를 바탕으로 다시 생각하고 행동을 이어갈 수 있게 합니다. 예를 들어, 다음과 같은 과정이 이뤄집니다:

	1.	사용자가 “안중근 의사랑 세종대왕이랑 이순신 장군 알려줘”라고 요청할 경우,
	2.	ReAct 모델은 먼저 문제를 생각하여 “안중근 의사부터 검색하고, 이후에 순차적으로 검색하는 것이 좋겠다”고 판단합니다.
	3.	그 후 행동으로 검색 도구를 호출하여 “안중근 의사”를 검색하고, 그 결과를 바탕으로 다시 생각과 행동을 반복해 나머지 인물들도 검색합니다.

이처럼 ReAct는 한번의 질문에 많은 컨텍스트가 혼재되어있는 경우 모델이 생각과 행동을 번갈아 수행하도록 하여, 복잡한 문제도 여러 단계를 거쳐 해결할 수 있도록 지원합니다. 이를 통해 LLM은 인간처럼 상황을 이해하고 문제를 해결하는 에이전트로 기능할 수 있게 됩니다.

In [1]:
%%capture
# update or install the necessary libraries
# !pip install --upgrade openai
%pip install --upgrade openai
%pip install --upgrade langchain
%pip install --upgrade python-dotenv
%pip install google-search-results
%pip install langchain_community
%pip install ChatOpenAI

In [2]:
import openai
import os

from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from dotenv import load_dotenv

load_dotenv()

False

In [3]:
import sys

sys.path.append(os.path.abspath("../../data"))

In [4]:
import API_KEY

SerpAPI_key = API_KEY.SerpAPI_KEY
os.environ["SERPAPI_API_KEY"] = SerpAPI_key

# ReAct 적용

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_15767/895470827.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-4o-mi...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [16]:
# 도구 설정. 구글링 API인 SerpAPI와 수학 계산을 위한 llm-math 도구를 가져옴
# tool customizing : https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [15]:
# model 과 tools 를 합쳐 agent 생성
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",  # langchain 에서 ReAct 를 사용할 수 있도록 agent 제공
    verbose=True,  # 무슨 생각을 하고 있는지 출력
    handle_parsing_errors=True,
)

# ReAct 기반 답변 생성

In [17]:
response = agent.run("지난주 프리미어리그 승리팀의 수와 오늘 서울의 기온을 더해줘")

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_45544/565429179.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("지난주 프리미어리그 승리팀의 수와 오늘 서울의 기온을 더해줘")




> Entering new AgentExecutor chain...
지난주 프리미어리그 승리팀의 수를 알아보려면 최근 경기 결과를 확인해야 하고, 오늘 서울의 기온도 확인해야 한다. 두 가지 정보를 모두 찾아보겠다. 

Action: Search  
Action Input: "프리미어리그 지난주 승리팀 수"  
Observation: ['1992년부터 현재까지 총 49팀이 프리미어리그 무대를 밟았고, 이 중 맨체스터 유나이티드 FC, 블랙번 로버스 FC, 아스널 FC, 첼시 FC, 맨체스터 시티 FC, 레스터 시티 FC, ...', '영국 현지 분석 "토트넘 브렌트포드 못 이긴다" (프리미어리그 5R 승부예측 총정리) #손흥민 #토트넘 #한준티비 한준TV 멤버십을 시작합니다.', 'epl 순위 결과 · 1위. 리버풀 · 2위. 맨시티 · 3위. 아스널 · 4위. 첼시 · 5위. 애스턴 빌라 · 6위. 브라이튼 · 7위. 뉴캐슬 · 8위. 풀럼.', '최다 우승: 맨체스터 유나이티드(20회) · 최다 연속 우승: 맨체스터 시티(4회) · 최다 참가: 에버튼, 아스날, 맨체스터 유나이티드, 리버풀, 첼시, 토트넘 ...', '매체는 "지난해 여름엔 부진했지만, 최근 맨체스터 시티전(2골)에서 최고의 경기를 펼친 뒤 리즈 유나이티드와 에버튼, 브라이튼전에서 팀 승리를 ...', '프리미어리그 2024/2025 실시간 점수와 최종 결과, 일정, 순위를 팔로우하세요! Livesport.com은 홈/어웨이/전체의 프리미어리그 2024/2025 순위와 최근 5경기성적, 오버/ ...', '손흥민(토트넘)이 결승골을 터뜨리며 팀을 승리로 이끈 가운데, EPL 순위가 지난주 13위에서 11계단 상승한 2위로 껑충 뛰어올랐다. 영국 스카이스포츠는 ...', '레스터와의 개막전에서 2-1로 승리를 거둔 맨유는 이제 11월에 열리는 리즈 유나이티드와의 조별리그 마지막 경기에서 승리를 거둬 본선 진출에 도전한다.', '"모두가 건강했다면, 그는 거기 없었을 것"이라고 뮌헨에서 열

In [18]:
response

'75'

# ReAct Agent + RAG

In [1]:
%pip install --quiet langchain tiktoken chromadb pymupdf sentence-transformers openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import initialize_agent

In [4]:
# PDF 파일로드
loader = PyMuPDFLoader(
    "../../data/nlp/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"
)
data = loader.load()

# 청킹
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(data)

In [5]:
# 임베딩 설정
embedding_model = "BAAI/bge-m3"

embd = HuggingFaceEmbeddings(model_name=embedding_model)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_11482/619414544.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embd = HuggingFaceEmbeddings(
/opt/anaconda3/envs/llm-env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
# 로컬 환경에서의 디렉토리 경로 설정 (예: 현재 작업 디렉토리 아래의 'vector' 폴더)
persist_directory = os.path.join(os.getcwd(), "vector")

# 벡터 스토어 생성
vectorstore = Chroma.from_documents(
    persist_directory=persist_directory,  # 저장할 디렉토리 경로
    documents=doc_splits,  # 문서 데이터
    embedding=embd,  # 임베딩 모델
)

# 리트리버로 변환
vectorstore_retriever = vectorstore.as_retriever()

In [10]:
vectorstore_retriever.get_relevant_documents("세계 인공시장에 대해 설명해줘")

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_11482/27759085.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  vectorstore_retriever.get_relevant_documents("세계 인공시장에 대해 설명해줘")


[Document(metadata={'author': 'kmd kdy', 'creationDate': "D:20220729090316+09'00'", 'creator': 'Hancom PDF 1.3.0.538', 'file_path': '../../data/nlp/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20220729090316+09'00'", 'page': 9, 'producer': 'Hancom PDF 1.3.0.538', 'source': '../../data/nlp/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'subject': '', 'title': '', 'total_pages': 18, 'trapped': ''}, page_content='직접적인 대체 기술이 없어 대체재로부터 위협이 낮은 편이며, 알고리즘의 동작원리를 이해하고 맞\n춤형 서비스를 지원하기 위한 솔루션 개발 능력이 뒷받침 되어야 하는 기술집약적 산업임\n▶시장조사전문기관 BCC research에 따르면 세계 인공지능 시장규모는 2020년 398.4억 달러에서 \n연평균 41.0% 성장하여 2025년에는 2,223.7억 달러의 시장을 형성할 것으로 전망됨\n￮'),
 Document(metadata={'author': 'kmd kdy', 'creationDate': "D:20220729090316+09'00'", 'creator': 'Hancom PDF 1.3.0.538', 'file_path': '../../data/nlp/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20220729090316+09'00'", 'page': 9, 'producer': 'Hancom PDF 1.3.0.538', 'source': '../../data/nlp/[이슈

In [12]:
# tool 생성
vectorstore_search_tool = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="ICT_industry_search",
    description="This PDF included information abotu trends in the ICT industry, including AI.",
)

tools = [vectorstore_search_tool]

# React Prompt
툴을 생성하면 일반적으로 ReACT 프롬프트에 다음과 같이 기록됩니다.
```
다음 질문에 최선을 다해 답변하세요. 당신이 사용할 수 있는 도구는 다음과 같습니다.:

ICT_industry_search: This PDF provides information about trends in the ICT industry, including AI.

질문: 당신이 답해야 할 입력 질문
생각: 무엇을 해야 할지 생각하세요
행동: 취해야 할 행동을 선택하세요. [ ICT_industry_search ] 중 하나여야 합니다.
행동 입력: 행동에 대한 입력을 작성하세요.
관찰: 행동의 결과를 작성하세요
...(이 생각/행동/행동 입력/관찰 단계는 여러 번 반복될 수 있습니다)
생각: 이제 최종 답변을 알겠습니다.
최종 답변: 원래 질문에 대한 최종 답변을 한국어로 작성하세요.

시작하세요!
```

즉 ReACT에서 기본적으로 사용할 Tool에 대한 설명(name과 Description)을 한 후 이 툴을 이용해 **행동**하도록 유도합니다.

여러 개의 툴을 사용하는 경우 다음과 같은 프롬프트가 기록됩니다. 계산기용 `Calculator` tool이 추가 됐다고 가정합니다.

툴을 생성하면 일반적으로 ReACT 프롬프트에 다음과 같이 기록됩니다.
```
다음 질문에 최선을 다해 답변하세요. 당신이 사용할 수 있는 도구는 다음과 같습니다.:

ICT_industry_search: This PDF provides information about trends in the ICT industry, including AI.
Calculator: 계산기입니다.

질문: 당신이 답해야 할 입력 질문
생각: 무엇을 해야 할지 생각하세요
행동: 취해야 할 행동을 선택하세요. [ ICT_industry_search, Calculator ] 중 하나여야 합니다.
행동 입력: 행동에 대한 입력을 작성하세요.
관찰: 행동의 결과를 작성하세요
...(이 생각/행동/행동 입력/관찰 단계는 여러 번 반복될 수 있습니다)
생각: 이제 최종 답변을 알겠습니다.
최종 답변: 원래 질문에 대한 최종 답변을 한국어로 작성하세요.

시작하세요!
```

즉 ReACT에서 기본적으로 사용할 Tool에 대한 설명(name과 Description)을 한 후 이 툴을 이용해 **행동**하도록 유도합니다.

In [13]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

/opt/anaconda3/envs/llm-env/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


## 모델 및 agent 생성

In [20]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

react_agent = create_react_agent(llm, tools=tools, prompt=prompt_react)

react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [21]:
react_agent_executor.invoke({"input": "세계 인공지능 시장규모를 년도 별로 설명해줘"})



> Entering new AgentExecutor chain...
세계 인공지능 시장규모에 대한 년도 별 정보를 찾기 위해 ICT 산업 관련 자료를 검색해야겠다. 
Action: ICT_industry_search
Action Input: '세계 인공지능 시장규모 년도 별' 세부 솔루션 분문별로는 2020년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3%를 차지할 
정도로 압도적으로 높음
[세계 인공지능 시장규모]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분
2020
2021
2022
2023
2024
2025
CAGR
(2020-2025)
인공지능
398.4
553.3
769.7
1,134.3
1,498.9
2,223.7
41.0%
　
(38.9)
(39.1)
(47.4)
(32.1)
(48.4)
  
소프트웨어
311.8
432.3
600.3
882.4
1,164.6
1,723.5
40.8%
　
(38.6)
(38.8)
(47.0)
(32.0)
(48.0)
  
서비스
55.7
78.0
109.3
162.6
216.0
323.7
42.2%
　
(40.0)

직접적인 대체 기술이 없어 대체재로부터 위협이 낮은 편이며, 알고리즘의 동작원리를 이해하고 맞
춤형 서비스를 지원하기 위한 솔루션 개발 능력이 뒷받침 되어야 하는 기술집약적 산업임
▶시장조사전문기관 BCC research에 따르면 세계 인공지능 시장규모는 2020년 398.4억 달러에서 
연평균 41.0% 성장하여 2025년에는 2,223.7억 달러의 시장을 형성할 것으로 전망됨
￮

(40.2)
(48.9)
(32.8)
(49.8)
  
하드웨어
30.9
43.1
60.2
89.2
118.3
176.5
41.7%
　
(39.5)
(39.7)
(48.2)
(32.5)
(49.2)
  
     자료: Global Artificial Intelligence(AI) Market, BCC Re

{'input': '세계 인공지능 시장규모를 년도 별로 설명해줘',
 'output': '세계 인공지능 시장규모는 다음과 같습니다 (단위: 억 달러):\n\n- 2020년: 398.4\n- 2021년: 553.3\n- 2022년: 769.7\n- 2023년: 1,134.3\n- 2024년: 1,498.9\n- 2025년: 2,223.7\n\n이 시장은 2020년부터 2025년까지 연평균 41.0% 성장할 것으로 전망됩니다. 소프트웨어 부문이 전체 시장의 78.3%를 차지하며, 서비스와 하드웨어 부문도 각각 성장하고 있습니다.'}

In [22]:
from langchain_core.prompts import PromptTemplate

kor_react_prompt_template = """다음 질문에 최선을 다해 답변하세요. 당신이 사용할 수 있는 도구는 다음과 같습니다.:

{tools}

다음 형식을 따릅니다:

Question: 당신이 답해야 할 입력 질문
Thought: 무엇을 해야 할지 생각하세요
Action: 취해야 할 행동을 선택하세요. [{tool_names}] 중 하나여야 합니다.
Action Input: 행동에 대한 입력을 작성하세요.
Observation: 행동의 결과를 작성하세요
...(이 생각/행동/행동 입력/관찰 단계는 여러 번 반복될 수 있습니다)
Thought: 이제 최종 답변을 알겠습니다.
Final Answer: 원래 질문에 대한 최종 답변을 작성하세요.

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

kor_react_prompt = PromptTemplate.from_template(kor_react_prompt_template)
kor_react_prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template='다음 질문에 최선을 다해 답변하세요. 당신이 사용할 수 있는 도구는 다음과 같습니다.:\n\n{tools}\n\n다음 형식을 따릅니다:\n\nQuestion: 당신이 답해야 할 입력 질문\nThought: 무엇을 해야 할지 생각하세요\nAction: 취해야 할 행동을 선택하세요. [{tool_names}] 중 하나여야 합니다.\nAction Input: 행동에 대한 입력을 작성하세요.\nObservation: 행동의 결과를 작성하세요\n...(이 생각/행동/행동 입력/관찰 단계는 여러 번 반복될 수 있습니다)\nThought: 이제 최종 답변을 알겠습니다.\nFinal Answer: 원래 질문에 대한 최종 답변을 작성하세요.\n\nBegin!\n\nQuestion: {input}\nThought: {agent_scratchpad}')

In [23]:
# Initialize ChatGroq model for language understanding
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create ReAct agent
react_agent = create_react_agent(
    llm, tools=tools, prompt=kor_react_prompt
)  # 한국어 프롬프트 지정

react_agent_executor = AgentExecutor(
    agent=react_agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [24]:
react_agent_executor.invoke({"input": "세계 인공지능 시장규모를 년도 별로 설명해줘"})



> Entering new AgentExecutor chain...
세계 인공지능 시장 규모에 대한 정보를 찾기 위해 ICT 산업 동향 문서를 검색해야 합니다.
Action: ICT_industry_search
Action Input: "세계 인공지능 시장 규모 년도별"세부 솔루션 분문별로는 2020년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3%를 차지할 
정도로 압도적으로 높음
[세계 인공지능 시장규모]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분
2020
2021
2022
2023
2024
2025
CAGR
(2020-2025)
인공지능
398.4
553.3
769.7
1,134.3
1,498.9
2,223.7
41.0%
　
(38.9)
(39.1)
(47.4)
(32.1)
(48.4)
  
소프트웨어
311.8
432.3
600.3
882.4
1,164.6
1,723.5
40.8%
　
(38.6)
(38.8)
(47.0)
(32.0)
(48.0)
  
서비스
55.7
78.0
109.3
162.6
216.0
323.7
42.2%
　
(40.0)

직접적인 대체 기술이 없어 대체재로부터 위협이 낮은 편이며, 알고리즘의 동작원리를 이해하고 맞
춤형 서비스를 지원하기 위한 솔루션 개발 능력이 뒷받침 되어야 하는 기술집약적 산업임
▶시장조사전문기관 BCC research에 따르면 세계 인공지능 시장규모는 2020년 398.4억 달러에서 
연평균 41.0% 성장하여 2025년에는 2,223.7억 달러의 시장을 형성할 것으로 전망됨
￮

(40.2)
(48.9)
(32.8)
(49.8)
  
하드웨어
30.9
43.1
60.2
89.2
118.3
176.5
41.7%
　
(39.5)
(39.7)
(48.2)
(32.5)
(49.2)
  
     자료: Global Artificial Intelligence(AI) Market, BCC Resear

{'input': '세계 인공지능 시장규모를 년도 별로 설명해줘',
 'output': '세계 인공지능 시장 규모는 2020년에 398.4억 달러였으며, 연평균 성장률(CAGR) 41.0%로 성장하여 2025년에는 2,223.7억 달러에 이를 것으로 전망됩니다. 각 년도별 시장 규모는 다음과 같습니다:\n- 2020년: 398.4억 달러\n- 2021년: 553.3억 달러\n- 2022년: 769.7억 달러\n- 2023년: 1,134.3억 달러\n- 2024년: 1,498.9억 달러\n- 2025년: 2,223.7억 달러\n\n이 정보는 BCC Research의 자료를 기반으로 하며, 인공지능 기술의 도입 증가와 스타트업의 증가가 시장 성장의 주요 촉진 요인으로 작용하고 있습니다.'}